In [4]:
import requests
import urllib.request
import time
from bs4 import BeautifulSoup
import pandas as pd
import re
import numpy as np
import json

In [207]:
baseUrl = 'https://www.nfl.com'
teamAbbr = pd.Series(["SF","CHI","CIN","BUF","DEN","CLE","TB","ARI","LAC","KC","IND","DAL","MIA","PHI","ATL","NYG",
                     "JAX","NYJ","DET","GB","CAR","NE","OAK","LA","BAL","WAS","NO","SEA","PIT","HOU","TEN","MIN"])
teamLocation = pd.Series(["San Francisco","Chicago","Cincinatti","Buffalo","Denver","Cleveland","Tampa Bay","Arizona",
                         "Los Angeles","Kansas City","Indianapolis","Dallas","Miami","Philadelphia","Atlanta","New York",
                         "Jacksonville","New York","Detroit","Green Bay","Carolina","New England","Oakland","Los Angeles",
                         "Baltimore","Washington","New Orleans","Seattle","Pittsburgh","Houston","Tennessee","Minnesota"])

In [6]:
playerPayload = {"type":"/stats","subject":"player-stats","category":"category",
                 "statisticCategory":"rushing","season":"2019",
                 "seasonType":"REG",
                 "list":"all",
                 "specificCategory":"rushingYards",
                 "order":"DESC"}

teamPayload = {"type":"/stats","subject":"team-stats","category":"offense",
               "statisticCategory":"passing",
               "season":"2019",
               "seasonType":"REG",
               "list":"all",
               "specificCategory":"",
               "order":""}

schedulePayload = {"type":"/schedules",
                  "season":"2019",
                  "seasonType":"REG"}


In [38]:
payload = teamPayload

url = 'https://www.nfl.com'
for key in payload:
    if payload[key]:
        url += payload[key]
        url += '/'
response = requests.get(url)
soup = BeautifulSoup(response.text,'html.parser')

In [39]:
columnHeader = []
header = soup.find('table').find_all('th')
for col in header:
    columnHeader.append(col.text)
columnHeader = [c.strip('\n') for c in columnHeader]
print(columnHeader)

['Team', 'Att', 'Cmp', 'Cmp %', 'Yds/Att', 'Pass Yds', 'TD', 'INT', 'Rate', '1st', '1st%', '20+', '40+', 'Lng', 'Sck', 'SckY']


In [54]:
season = soup.find_all('select', {"class":"d3-o-dropdown"})[1].find_all("option")
seasonWeeks = []
for week in season:
    seasonWeeks.append(week.get("value"))
current = soup.find_all('select', {"class":"d3-o-dropdown"})[1].find('option', {"selected":True}).get("value")
print(current)
seasonWeeks
for week in seasonWeeks:
    time.sleep(1)
    response = requests.get(baseUrl+week)
    print("Response:", response.status_code,response.url)

/schedules/2019/REG1/
Response: 200 https://www.nfl.com/schedules/2019/PRE0/
Response: 200 https://www.nfl.com/schedules/2019/PRE1/
Response: 200 https://www.nfl.com/schedules/2019/PRE2/
Response: 200 https://www.nfl.com/schedules/2019/PRE3/
Response: 200 https://www.nfl.com/schedules/2019/PRE4/
Response: 200 https://www.nfl.com/schedules/2019/REG1/
Response: 200 https://www.nfl.com/schedules/2019/REG2/
Response: 200 https://www.nfl.com/schedules/2019/REG3/
Response: 200 https://www.nfl.com/schedules/2019/REG4/


KeyboardInterrupt: 

In [12]:
tableRows = []
pagingText = True


while pagingText:
    time.sleep(2)
    response = requests.get(url)
    print("url:",response.url)
    soup = BeautifulSoup(response.text,'html.parser')
    table = soup.find('table')
    rows = table.find_all('tr')

    for row in rows:
        cols = row.find_all('td')
        if cols:
            cols = [ele.text.strip() for ele in cols]
            tableRows.append([ele for ele in cols if ele]) # Get rid of empty values
    pagingText = soup.find("a", {"class": "nfl-o-table-pagination__next"})
    if pagingText:
        url = baseUrl + pagingText.get('href')

url: https://www.nfl.com/schedules/2019/REG/


AttributeError: 'NoneType' object has no attribute 'find_all'

In [ ]:
resultsDf = pd.DataFrame(tableRows, columns=columnHeader)
# remove \n whitespace from team names
find = re.compile(r"^(\n*).*")
resultsDf['Team'] = resultsDf['Team'].apply(lambda x: re.search(find,x).group())
pd.DataFrame.head(resultsDf)

In [245]:
def getStats(url, payload): 
    tableRows = []
    pagingText = True
    
    for key in payload:
        if payload[key]:
            url += payload[key]
            url += '/'
    response = requests.get(url)
    
    # get column names
    soup = BeautifulSoup(response.text,'html.parser')
    columnHeader = []
    header = soup.find('table').find_all('th')
    for col in header:
        columnHeader.append(col.text)
    columnHeader = [c.strip('\n') for c in columnHeader]

    # search through pages and append to tableRows
    while pagingText:
        time.sleep(2)
        response = requests.get(url)
        print("url:",response.url)
        soup = BeautifulSoup(response.text,'html.parser')
        table = soup.find('table')
        rows = table.find_all('tr')

        for row in rows:
            cols = row.find_all('td')
            if cols:
                cols = [ele.text.strip() for ele in cols]
                cols = [ele.rstrip('\n') for ele in cols]
                tableRows.append([ele for ele in cols if ele]) # Get rid of empty values
        pagingText = soup.find("a", {"class": "nfl-o-table-pagination__next"})
        if pagingText:
            url = baseUrl + pagingText.get('href')
    
    resultsDf = pd.DataFrame(tableRows, columns=columnHeader)
    find = re.compile(r"^(\n*).*")
    resultsDf['Team'] = resultsDf['Team'].apply(lambda x: re.search(find,x).group())
    resultsDf = resultsDf.sort_values(by=['Team']).reset_index(drop=True)
    return resultsDf

In [246]:
teamPayload = {"stats":"/stats","subject":"team-stats",
               "category":"offense",
               "statisticCategory":"passing",
               "season":"2019",
               "seasonType":"REG",
               "list":"all",
               "specificCategory":"",
               "order":""}


In [274]:
def getTeamStats(url,side,categ,season,cols):
    allCols = ['Team','weekType']+cols
    seasonTypes = ["PRE","REG","POST"]
    df = None
    for st in seasonTypes:
        teamPayload = {"stats":"/stats","subject":"team-stats",
               "category":side,
               "statisticCategory":categ,
               "season":season,
               "seasonType":st,
               "list":"all"}
        tempDf = getStats(baseUrl,teamPayload)
        tempDf['weekType']=st
        tempDf = tempDf[allCols] 
        
        # change raw stats to ordering
        for i in cols:
            colName = i + '(o)'
            temp = pd.to_numeric(tempDf[i])
            tempDf[colName] = temp
            tempDf = tempDf.sort_values(by=i).reset_index(drop=True)
            tempDf[colName] = range(len(tempDf.index),0,-1)
            temp = tempDf[colName].astype(str)
            tempDf[colName] = temp
            
        df = pd.concat([df,tempDf])
    return df

In [275]:
poStats = ['Pass Yds','Yds/Att','Cmp %','TD','INT']
roStats = ['Rush Yds','YPC','TD','Rush FUM']
pdStats = ['Yds','Yds/Att','Cmp %','TD','INT']
rdStats = ['Rush Yds','YPC','TD','Rush FUM']

teamPODf = getTeamStats(baseUrl,"offense","passing","2019",poStats)
teamRODf = getTeamStats(baseUrl,"offense","rushing","2019",roStats)
teamPDDf = getTeamStats(baseUrl,"defense","passing","2019",pdStats)
teamRDDf = getTeamStats(baseUrl,"defense","rushing","2019",rdStats)
print('done')

url: https://www.nfl.com/stats/team-stats/offense/passing/2019/pre/all
url: https://www.nfl.com/stats/team-stats/offense/passing/2019/reg/all
url: https://www.nfl.com/stats/team-stats/offense/passing/2019/post/all
url: https://www.nfl.com/stats/team-stats/offense/rushing/2019/pre/all
url: https://www.nfl.com/stats/team-stats/offense/rushing/2019/reg/all
url: https://www.nfl.com/stats/team-stats/offense/rushing/2019/post/all
url: https://www.nfl.com/stats/team-stats/defense/passing/2019/pre/all
url: https://www.nfl.com/stats/team-stats/defense/passing/2019/reg/all
url: https://www.nfl.com/stats/team-stats/defense/passing/2019/post/all
url: https://www.nfl.com/stats/team-stats/defense/rushing/2019/pre/all
url: https://www.nfl.com/stats/team-stats/defense/rushing/2019/reg/all
url: https://www.nfl.com/stats/team-stats/defense/rushing/2019/post/all
done


In [284]:
teamPODf.loc[teamPODf['weekType']=='REG'].sort_values(by=['TD'])

,Team,weekType,Pass Yds,Yds/Att,Cmp %,TD,INT,Pass Yds(o),Yds/Att(o),Cmp %(o),TD(o),INT(o)
0,Broncos,REG,3401,6.7,61.9,16,10,28,23,21,32,32
19,Panthers,REG,4134,6.5,60.3,17,21,10,29,27,31,13
6,Redskins,REG,3205,6.7,62.2,18,13,32,25,17,28,26
16,Steelers,REG,3214,6.3,61.8,18,19,31,31,22,29,16
11,Bengals,REG,3994,6.5,57.8,18,16,20,28,32,30,21
12,Jets,REG,3443,6.6,62,19,16,27,27,19,27,20
3,Cardinals,REG,3797,6.9,64.1,20,12,23,18,13,25,29
4,Bears,REG,3573,6.2,64,20,12,25,32,14,26,28
5,Bills,REG,3476,6.8,58.3,21,12,26,20,31,24,27
29,Raiders,REG,4110,7.9,70.2,22,8,13,8,2,22,3


In [285]:
dfs = [teamPODf,teamRODf,teamPDDf,teamRDDf]
for i in range(len(dfs)):
    with open("teamStat"+str(i),"w") as outfile:
        outfile.write(dfs[i].to_json(orient='records'))

In [193]:
teamPDDf.sort_values(by=['Team','weekType'])

,Yds,Yds/Att,Cmp %,TD,INT
12,32,32,26,18,20
13,26,28,26,30,17
12,32,32,26,18,20
4,24,28,15,29,28
26,12,5,11,3,4
...,...,...,...,...,...
22,4,3,13,10,8
24,9,16,10,15,8
28,18,23,6,19,4
5,21,24,16,28,25


In [194]:
teamRDDf.sort_values(by=['Team','weekType'])

,Rush Yds,YPC,TD,Rush FUM
27,16,11,26,5
3,12,4,5,27
27,16,11,26,5
9,24,27,7,23
25,6,3,10,5
...,...,...,...,...
13,20,23,16,17
10,21,26,15,22
1,20,14,30,31
1,16,6,28,29


In [1]:
def getMatchups(url,payload):
    for key in payload:
        if payload[key]:
            url += payload[key]
            url += '/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    
    matchups = []
    columnHeader = ['matchupId', 'team1', 'team2', 'team1Score','team2Score','weekType','weekNum']
    season = soup.find_all('select', {"class":"d3-o-dropdown"})[1].find_all("option")
    seasonWeeks = []
    for week in season:
        seasonWeeks.append(week.get("value"))
    for week in seasonWeeks:
        time.sleep(2)
        url = baseUrl + week
        response = requests.get(url)
        print(url)
        soup = BeautifulSoup(response.text,'html.parser')
        weekName = week.rsplit('/', 2)[-2]
        weekType = re.findall("[a-zA-Z]+", weekName)[0]
        weekNum = re.findall(r'\d+', weekName)[0]
        games = soup.find_all('a', {"class": "nfl-c-matchup-strip__game"})
        index = 0
        for game in games:
            gameSoup = game.find_all('span', {"class":"nfl-c-matchup-strip__team-abbreviation"})
            matchup = []
            matchup.append(str(index))
            for team in gameSoup:
                matchup.append(re.sub(r'[^A-Za-z]', '', team.get_text()))
            scoreSoup = game.find_all('div', {'class':"nfl-c-matchup-strip__team-score"})
            if scoreSoup:
                for team in scoreSoup:
                    matchup.append(team.get_text())
            else:
                matchup.append("")
                matchup.append("")
            matchup.append(weekType)
            matchup.append(weekNum)
            matchups.append(matchup)
            index += 1
    matchupsDf = pd.DataFrame(matchups,columns=columnHeader)
    return matchupsDf

In [7]:
schedulePayload = {"type":"/schedules",
                  "season":"2019",
                  "seasonType":"REG"}
df2 = getMatchups(baseUrl,schedulePayload)

#matchupsDf.loc[(matchupsDf['weekType']=='REG') & (matchupsDf['weekNum']=='1')]

https://www.nfl.com/schedules/2019/PRE0/
https://www.nfl.com/schedules/2019/PRE1/
https://www.nfl.com/schedules/2019/PRE2/
https://www.nfl.com/schedules/2019/PRE3/
https://www.nfl.com/schedules/2019/PRE4/
https://www.nfl.com/schedules/2019/REG1/
https://www.nfl.com/schedules/2019/REG2/
https://www.nfl.com/schedules/2019/REG3/
https://www.nfl.com/schedules/2019/REG4/
https://www.nfl.com/schedules/2019/REG5/
https://www.nfl.com/schedules/2019/REG6/
https://www.nfl.com/schedules/2019/REG7/
https://www.nfl.com/schedules/2019/REG8/
https://www.nfl.com/schedules/2019/REG9/
https://www.nfl.com/schedules/2019/REG10/
https://www.nfl.com/schedules/2019/REG11/
https://www.nfl.com/schedules/2019/REG12/
https://www.nfl.com/schedules/2019/REG13/
https://www.nfl.com/schedules/2019/REG14/
https://www.nfl.com/schedules/2019/REG15/
https://www.nfl.com/schedules/2019/REG16/
https://www.nfl.com/schedules/2019/REG17/
https://www.nfl.com/schedules/2019/POST1/
https://www.nfl.com/schedules/2019/POST2/
https:

In [26]:
pd.DataFrame.head(df2,50)

,matchupId,team1,team2,team1Score,team2Score,weekType,weekNum
0,0,DEN,ATL,14,10,PRE,0
1,0,NYJ,NYG,22,31,PRE,1
2,1,IND,BUF,16,24,PRE,1
3,2,TEN,PHI,27,10,PRE,1
4,3,ATL,MIA,27,34,PRE,1
5,4,NE,DET,31,3,PRE,1
6,5,WAS,CLE,10,30,PRE,1
7,6,JAX,BAL,0,29,PRE,1
8,7,HOU,GB,26,28,PRE,1
9,8,CAR,CHI,23,13,PRE,1


In [11]:
matchupsJson = df2.to_json(orient='records')

with open("matchups.json","w") as outfile:
    outfile.write(matchupsJson)

In [65]:
weekTypeD = {"PRE":"Pre-Season", "REG":"Regular Season", "POST": "Post-Season"}


def getWeeks(url,payload):
    for key in payload:
        if payload[key]:
            url += payload[key]
            url += '/'
    response = requests.get(url)
    soup = BeautifulSoup(response.text,'html.parser')
    columnHeader = ['weekType','weekNum','weekTypeF','weekF']
    matchups = []
    season = soup.find_all('select', {"class":"d3-o-dropdown"})[1].find_all("option")
    seasonWeeks = []
    for week in season:
        seasonWeeks.append(week.get("value"))
    for week in seasonWeeks:
        url = baseUrl + week
        weekName = week.rsplit('/', 2)[-2]
        weekType = re.findall("[a-zA-Z]+", weekName)[0]
        weekNum = re.findall(r'\d+', weekName)[0]
        if weekType in weekTypeD.keys():
            weekTypeF = weekTypeD[weekType]
            weekF = weekTypeF + " Week " + weekNum
            matchup = []
            matchup.append(weekType)
            matchup.append(weekNum)
            matchup.append(weekTypeF)
            matchup.append(weekF)
            matchups.append(matchup)
    weeksDf = pd.DataFrame(matchups,columns=columnHeader)
    return weeksDf

In [66]:
schedulePayload = {"type":"/schedules",
                  "season":"2018",
                  "seasonType":"REG"}
df3 = getWeeks(baseUrl,schedulePayload)
df3
weeksJson = df3.to_json(orient='records')

with open("weeks.json","w") as outfile:
    outfile.write(weeksJson)